# **Tidying**

Part of the data engineering process is data cleaning and tidying. What is done in those two processes, is trying to make the data more readable, and complete. This makes much easier to analyze, visualize, and train the data.


## **Data Tidying**

Making the data more organized, and readable is the result of applying data tidying. 

In this section two main pandas functions are used in data tidying those are `melt` and `pivot_table`.

Let's start by taking a look at the below dataframe, which represents the income ranges based on religion. This is part of the PEW research, which is famous in the US for conducting pollings and surveys on citizens.

When the following are satisfied:


1. Each variable forms a column
2. Each observation forms a row
3. Each type of observational unit forms a table

We can then say that our dataset is *tidy*.

First we need to import pandas to read csv datasets.

In [ ]:
import pandas as pd

# Load the dataset
pew_data = pd.read_csv('data/pew-raw.csv')

# Inpect the dataset
print("First five rows of the dataset:")
print(pew_data.head())

print("Access specific row using iloc:[0]")
print(pew_data.iloc[0])

print("Access specific row using loc:[0]")
print(pew_data.loc[0, 'religion'])

pew_tidy = pew_data.melt(id_vars='religion', var_name='income', value_name='count')

print("Tidy dataset:")
print(pew_tidy.head())

First five rows of the dataset:
            religion   <$10k   $10-20k  $20-30k  $30-40k   $40-50k  $50-75k
0           Agnostic      27        34       60       81        76      137
1            Atheist      12        27       37       52        35       70
2           Buddhist      27        21       30       34        33       58
3           Catholic     418       617      732      670       638     1116
4  Dont know/refused      15        14       15       11        10       35
Access specific row using iloc:[0]
religion    Agnostic
 <$10k            27
 $10-20k          34
$20-30k           60
$30-40k           81
 $40-50k          76
$50-75k          137
Name: 0, dtype: object
Access specific row using loc:[0]
Agnostic
Tidy dataset:
            religion  income  count
0           Agnostic   <$10k     27
1            Atheist   <$10k     12
2           Buddhist   <$10k     27
3           Catholic   <$10k    418
4  Dont know/refused   <$10k     15


### **PEW Research Dataset**

1. Start by Importing the dataset into a pandas dataframe.

2. Observe the dataset using the `loc`, `iloc`, `head`, or `tail` approaches

***What does not seem right in the above dataframe?***

3. Try to make the column headers represent a variable not a value. For that, use the `melt` function.

### **Billboard Dataset**

This dataset outlines data about the top hit songs on the Billboard list and the week from entrance that it was in the billboard with the ranking.

1. Read the dataset and store it in a pandas dataframe. Note that the usual utf-8 encoding does not work on this dataset. The reason behind this is that there might be characters that are not supported by `utf-8`.

The suggestion is to use for this dataset `unicode_escape` encoding. (converts all non-ASCII characters into their \uXXXX representations)

2. Observe the first few rows of the dataset.

***What is wrong with the above dataset?***

3. Let's, again, use the `melt` function to fix the general structure of the dataframe.

If we inspect the current dataframe. We find that it is structured in a better way than before. 

However, the ***Week*** column looks a bit ugly!

4. Let's try to place only the week number in that column without the extras surronding it.

5. Now let's inspect the ***Week*** column in the dataframe.

Next, let's try to find the date at which the song ranked the number that is shown per row.

6. To do that let's first think of the equation that is going to get us the relevant date at which the song ranked the *rth*.



*Timedeltas are absolute differences in times, expressed in difference units (e.g. days, hours, minutes, seconds). This method converts an argument from a recognized timedelta format / value into a Timedelta type.*


***What is the problem with the calculation above?***

7. Let's only keep necessary columns

8. How to rename your columns?

Display the dataframe

In the above dataframe, there are some *NaN* values. What are we going to do? <br/>
9. Apply quick data cleaning and then observe the dataset

In [7]:
import pandas as pd

# 1. Load the dataset using `unicode_escape` encoding
billboard = pd.read_csv("data/billboard.csv", encoding="unicode_escape")

# 2. Observe the first few rows
print("Initial view of the dataset:")
print(billboard.head())

# 3. Melt the dataset — gather all week columns
billboard_melted = billboard.melt(
    id_vars=["year", "artist.inverted", "track", "time", "genre", "date.entered", "date.peaked"],
    var_name="week",
    value_name="rank"
)

# 4. Clean the week column (remove 'x' and '.week')
billboard_melted["week"] = billboard_melted["week"].str.extract(r'(\d+)').astype(int)

# 5. Inspect cleaned week column
print("\nCleaned 'week' column sample:")
print(billboard_melted["week"].unique()[:10])

# 6. Compute charting date: date.entered + week offset
# Convert date.entered to datetime
billboard_melted["date.entered"] = pd.to_datetime(billboard_melted["date.entered"], errors="coerce")

# Add timedelta of (week - 1) weeks to date.entered
billboard_melted["chart.date"] = billboard_melted["date.entered"] + pd.to_timedelta((billboard_melted["week"] - 1) * 7, unit="d")

# What could go wrong here?
# ➤ If `date.entered` is missing or invalid, chart.date will be NaT

# 7. Keep only necessary columns
columns_to_keep = ["artist.inverted", "track", "genre", "week", "rank", "chart.date"]
billboard_cleaned = billboard_melted[columns_to_keep]

# 8. Rename columns
billboard_cleaned = billboard_cleaned.rename(columns={
    "artist.inverted": "artist",
    "track": "title",
    "genre": "genre",
    "week": "week_number",
    "rank": "weekly_rank",
    "chart.date": "ranking_date"
})

# 9. Drop rows with NaN and display final result
billboard_cleaned.dropna(inplace=True)
print("\nFinal cleaned Billboard dataset:")
print(billboard_cleaned.head(400))


Initial view of the dataset:
   year      artist.inverted                                  track  time  \
0  2000      Destiny's Child               Independent Women Part I  3:38   
1  2000              Santana                           Maria, Maria  4:18   
2  2000        Savage Garden                     I Knew I Loved You  4:07   
3  2000              Madonna                                  Music  3:45   
4  2000  Aguilera, Christina  Come On Over Baby (All I Want Is You)  3:38   

  genre date.entered date.peaked  x1st.week  x2nd.week  x3rd.week  ...  \
0  Rock   2000-09-23  2000-11-18         78       63.0       49.0  ...   
1  Rock   2000-02-12  2000-04-08         15        8.0        6.0  ...   
2  Rock   1999-10-23  2000-01-29         71       48.0       43.0  ...   
3  Rock   2000-08-12  2000-09-16         41       23.0       18.0  ...   
4  Rock   2000-08-05  2000-10-14         57       47.0       45.0  ...   

   x67th.week  x68th.week  x69th.week  x70th.week  x71st.week  

## **Data Cleaning**

Data cleaning involves removing unwanted characters, imputing, or dropping missing values.

The decision is based on the dataset you have, and the information you can extract from the other columns.


Examples of data cleaning include cleaning:

1.   **Missing Data**
2.   **Irregular Data** (Outliers)
3.   **Unnecessary Data** — Repetitive Data, Duplicates and more
4.   **Inconsistent Data** — Capitalization, Addresses and more








### **Cars Data Set**

Start by reading the dataset related to car models: ./CSVs/cars.csv

In [13]:
import pandas as pd

# 1. Load the dataset (skip the first metadata row and use ';' as separator)
cars = pd.read_csv("data/cars.csv", sep=";", skiprows=[1])

# 2. View the first few rows
print("Initial dataset preview:")
print(cars.head())

# 3. Check for missing values
print("\nMissing values per column:")
print(cars.isna().sum())

# 4. Convert relevant columns to numeric (force coercion of invalid strings)
cars["MPG"] = pd.to_numeric(cars["MPG"], errors="coerce")
cars["Horsepower"] = pd.to_numeric(cars["Horsepower"], errors="coerce")
cars["Displacement"] = pd.to_numeric(cars["Displacement"], errors="coerce")
cars["Weight"] = pd.to_numeric(cars["Weight"], errors="coerce")

# 5. Handle missing data
cars.fillna({"MPG":cars["MPG"].median()}, inplace=True)
cars.fillna({"Horsepower":cars["Horsepower"].median()}, inplace=True)
cars.fillna({"Displacement":cars["Displacement"].median()}, inplace=True)
cars.fillna({"Weight":cars["Weight"].median()}, inplace=True)

# 6. Drop any rows still containing NaN
cars.dropna(inplace=True)

# 7. Remove duplicate entries
cars_cleaned = cars.drop_duplicates()

# 8. Final dataset info
print("\nCleaned dataset info:")
print(cars_cleaned.info())

# 9. Preview cleaned data
print("\nCleaned data sample:")
print(cars_cleaned.head())


Initial dataset preview:
                         Car   MPG  Cylinders  Displacement  Horsepower  \
0  Chevrolet Chevelle Malibu   NaN          8         307.0       130.0   
1          Buick Skylark 320  15.0          8         350.0         NaN   
2         Plymouth Satellite   NaN          8         318.0       150.0   
3              AMC Rebel SST  16.0          8           NaN       150.0   
4                Ford Torino  17.0          8         302.0       140.0   

   Weight  Acceleration  Model Origin  
0  3504.0          12.0     70     US  
1  3693.0          11.5     70     US  
2  3436.0          11.0     70     US  
3     NaN          12.0     70     US  
4  3449.0          10.5     70     US  

Missing values per column:
Car             0
MPG             3
Cylinders       0
Displacement    1
Horsepower      2
Weight          1
Acceleration    0
Model           0
Origin          0
dtype: int64

Cleaned dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entr

First Row seems to be the datatype, we need to remove it

Let's observe the columns with null values.  Either by using the `isnull().sum()` function 

There aren't many missing values. Let's take a glimpse at the percentage of the missing values:

**HINT:** We'll need `Numpy` for the below task.

Around *0.19%* of the values are missing, which isn't a lot. Therefore, we might go with the option of dropping all the rows with null values.

Lets also check dropping the columns

Let's observe how many columns we lost

### **Cars Dataset - Filling in missing values automatically**

Another option is to try and fill in the missing values through imputations.

Let's take the `MPG` column for example. We can fill in the missing values with 0s through the following line of code:

`df_cars.fillna(0) `. 

However, this does not make much sense as there isn't MPG equal to 0. How about we plot the MPG column and if it follows a random distribution we can use the mean of the column to compute the missing values. Otherwise, we can use the median (if there is a skewed normal distribution). However, there might be a better way of imputation which is getting the median or the mean of the MPG of the cars with similar attributes.

If we observe the graph above, we can consider it in a way or another normally distributed. Therefore, we can impute the missing values using the mean.

To compute the mean we need numeric values. However the values in the dataframe are objects. Therefore, we need to change them to numerics so that we can compute them.

Now let's see what is the mean of the MPG column

We can use this mean to compute the missing values since the graph demonstarted a normal distribution

### **Car Dataset - Simple Imputer**

*SimpleImputer* is a `scikit-learn` class which is helpful in handling the missing data in the predictive model dataset. It replaces the `NaN` values with a specified placeholder.
It is implemented by the use of the `SimpleImputer()` method which takes the following arguments :

`missing_values` : The missing_values placeholder which has to be imputed. By default is NaN

`strategy` : The data which will replace the NaN values from the dataset. The strategy argument can take the values – ‘mean'(default), ‘median’, ‘most_frequent’ and ‘constant’.


Let's start by importing the `SimpleImputer` into our notebook

What we need to do are two essential steps:

1. fit the data (compute the mean / median / most freq)
2. transform the data (place the computed values in the NaN cells)

In [12]:
import pandas as pd
from sklearn.impute import SimpleImputer

# Load dataset (skip data type row)
cars = pd.read_csv("data/cars.csv", sep=";", skiprows=[1])

# Convert relevant columns to numeric
numeric_cols = ["MPG", "Displacement", "Horsepower", "Weight", "Acceleration"]
for col in numeric_cols:
    cars[col] = pd.to_numeric(cars[col], errors="coerce")

# Create an imputer instance with median strategy
imputer = SimpleImputer(strategy="median")

# Apply imputer to numeric columns
cars[numeric_cols] = imputer.fit_transform(cars[numeric_cols])

# Drop remaining NaNs (if any) and remove duplicates
cars.dropna(inplace=True)
cars_cleaned = cars.drop_duplicates().reset_index(drop=True)

print("Final cleaned data using SimpleImputer:")
print(cars_cleaned.head())


Final cleaned data using SimpleImputer:
                         Car   MPG  Cylinders  Displacement  Horsepower  \
0  Chevrolet Chevelle Malibu  22.5          8         307.0       130.0   
1          Buick Skylark 320  15.0          8         350.0        92.5   
2         Plymouth Satellite  22.5          8         318.0       150.0   
3              AMC Rebel SST  16.0          8         151.0       150.0   
4                Ford Torino  17.0          8         302.0       140.0   

   Weight  Acceleration  Model Origin  
0  3504.0          12.0     70     US  
1  3693.0          11.5     70     US  
2  3436.0          11.0     70     US  
3  2815.0          12.0     70     US  
4  3449.0          10.5     70     US  


## **Outlier Detection** 


An Outlier is a data-item/object that deviates significantly from the rest of the (so-called normal)objects. They can be caused by measurement or execution errors. The analysis for outlier detection is referred to as outlier mining. There are many ways to detect the outliers, and the removal process is the data frame same as removing a data item from the panda’s data frame.



https://www.geeksforgeeks.org/detect-and-remove-the-outliers-using-python/





### Outliers Visualization

#### Visualizing Outliers Using Box Plot
It captures the summary of the data effectively and efficiently with only a simple box and whiskers. Boxplot summarizes sample data using 25th, 50th, and 75th percentiles. One can just get insights(quartiles, median, and outliers) into the dataset by just looking at its boxplot.

#### Visualizing Outliers Using ScatterPlot.

It is used when you have paired numerical data and when your dependent variable has multiple values for each reading independent variable, or when trying to determine the relationship between the two variables. In the process of utilizing the scatter plot, one can also use it for outlier detection.

#### Z-Score:
Z- Score is also called a standard score. This value/score helps to understand that how far is the data point from the mean. And after setting up a threshold value one can utilize z score values of data points to define the outliers.
<br>
Zscore = (data_point -mean) / std. deviation



Now to define an outlier threshold value is chosen which is generally 3.0. As 99.7% of the data points lie between +/- 3 standard deviation (using Gaussian Distribution approach).

Rows where Z value is greater than 2

#### IQR (Inter-Quartile Range)
Inter Quartile Range approach to finding the outliers is the most commonly used and most trusted approach used in the research field. <Br>
IQR = Quartile3 - Quartile1

To define the outlier base value is defined above and below dataset’s normal range namely Upper and Lower bounds, define the upper and the lower bound (1.5*IQR value is considered) :<br>
upper = Q3 + 1.5 * IQR <br>
lower = Q1 - 1.5 * IQR <br> 

- Removing the outliers:
For removing the outlier, one must follow the same process of removing an entry from the dataset using its exact position in the dataset because in all the above methods of detecting the outliers end result is the list of all those data items that satisfy the outlier definition according to the method used.